In [263]:
import numpy as np
import math
import residuals as r
np.set_printoptions(threshold=40)
np.set_printoptions(suppress=True)

In [379]:
# I am defining the weighted standard deviation according to this definition:
# https://en.wikipedia.org/wiki/Weighted_arithmetic_mean#Weighted_sample_variance
# I am using the inverse square of the error arrays to define the weights

factor10 = math.sqrt((25*60)/10.0) 
factor80 = math.sqrt((25*60)/80.0) 



def weighted_stddev(values, errors):
    '''
    Return the weighted average and standard deviation.

    '''
    weights = 1 / ((errors)**2)
    average = np.average(values, weights=weights)
    variance = np.average((values-average)**2, weights=weights)  # Fast and numerically precise
    return (math.sqrt(variance))

def magnitude(x):
    var = float(x)
    return int(math.floor(math.log10(var)))

def jitter_ratio(resids_file, condition_lower, condition_upper, titlearg):
    '''
    Return the ratio of sigma_dump over sigma_all.

    '''
    
    #Define sigma_all (or std_all) as entire observation period
    x=r.read_residuals(filename=resids_file)

    condition = (x.bary_TOA > condition_lower) & (x.bary_TOA < condition_upper)
    Resid_all = x.prefit_sec[condition]
    Errors_all = x.uncertainty[condition]
    std_all = weighted_stddev(Resid_all, Errors_all)
    
    ##============
    
    #Define sigma_dump (or std_all) as individual dumps within observation period
    
    #This isolates the numbers for the different dump times
    rounded = np.round(np.array([x.bary_TOA[condition] - 0.00005]), 4)
    dumptimes=[]
    for i in rounded.tolist()[0]: 
        if i not in dumptimes:
            dumptimes.append(i)

    #This calculates a list of sigma_dump values and averages those
    str_dump_devs=[]
    for i in dumptimes:
        timeindex = (rounded == i)[0]
        Resid = Resid_all[timeindex]
        Errors = Errors_all[timeindex]
        single_std_dump = weighted_stddev(Resid, Errors)
        str_dump_devs.append(single_std_dump)
    
    #to convert str_dump_devs out of scientific notation
    dump_devs=[]
    for i in str_dump_devs:
        dump_devs.append('{0:.20f}'.format(i))
        
    #prints the magnitude of each sigma_dump in a list and finds average magnitude
    magarray=[]
    for i in dump_devs:
        magarray.append(magnitude(i))
    averagemag = round(np.mean(magarray))
    
    arg=averagemag+1.5
    makemask = np.ma.masked_where(np.array(magarray) > arg, np.array(magarray))
    
    std_dump=np.mean(str_dump_devs)
    masked_std_dump = np.mean(np.ma.masked_where(np.ma.getmask(makemask), str_dump_devs))
        
#     for i in dumptimes:
#         timeindex = (rounded == i)[0]
#         Errors = Errors_all[timeindex]
#         print Errors
    
    #Setup jitter ratio
    ratio = std_all / std_dump
    masked_ratio = std_all / masked_std_dump
    
    print titlearg
    print "sigma_all = " + str(std_all)
    print "average sigma_dump = " + str(std_dump)
#     print "every sigma_dump = " + str(dump_devs)
    print "jitter ratio = " + str(ratio) 
    print "masked jitter ratio = " + str(masked_ratio) 

In [92]:
#Checking order of operations

print 25*60
print 1500/10
print math.sqrt(150)
print math.sqrt((25*60)/10)

print math.sqrt((25*60)/80)

#yes, order of operations works

1500
150
12.2474487139
12.2474487139
4.24264068712


In [382]:
jitter_ratio("/nimrod1/eschwab/residuals/J1713+0747_resid_56380_80F8.tmp",
            56380.35, 56380.367, 'J1713+0747, MJD 56380, 80 second subintervals')

J1713+0747, MJD 56380, 80 second subintervals
sigma_all = 2.44415585816e-07
average sigma_dump = 2.49624000145e-08
jitter ratio = 9.79134961678
masked jitter ratio = 9.79134961678


In [383]:
jitter_ratio("/nimrod1/eschwab/residuals/J1713+0747_resid_56380_NTF8.tmp",
            56380.35, 56380.367, 'J1713+0747, MJD 56380, 10 second subintervals')

J1713+0747, MJD 56380, 10 second subintervals
sigma_all = 5.82671845721e-07
average sigma_dump = 7.10113781891e-08
jitter ratio = 8.20533075938
masked jitter ratio = 8.20533075938


In [384]:
jitter_ratio("/nimrod1/eschwab/residuals/J1713+0747_resid_56201_80F8.tmp",
            56201.85, 56201.872, 'J1713+0747, MJD 56201, 80 second subintervals')

J1713+0747, MJD 56201, 80 second subintervals
sigma_all = 1.73501235065e-07
average sigma_dump = 5.08948679605e-08
jitter ratio = 3.40901238215
masked jitter ratio = 3.40901238215


In [385]:
jitter_ratio("/nimrod1/eschwab/residuals/J1713+0747_resid_56201_NTF8.tmp",
            56201.85, 56201.872, 'J1713+0747, MJD 56201, 10 second subintervals')

J1713+0747, MJD 56201, 10 second subintervals
sigma_all = 5.43224438005e-07
average sigma_dump = 8.13611454109e-08
jitter ratio = 6.67670588045
masked jitter ratio = 6.67670588045


In [387]:
jitter_ratio("/nimrod1/eschwab/residuals/B1937+21_resid_56465_80F8.tmp",
            56465.3, 56465.32, 'B1937+21, MJD 56465, 80 second subintervals')

B1937+21, MJD 56465, 80 second subintervals
sigma_all = 4.68059107539e-08
average sigma_dump = 4.44210286622e-08
jitter ratio = 1.05368813293
masked jitter ratio = 1.05368813293


In [388]:
jitter_ratio("/nimrod1/eschwab/residuals/B1937+21_resid_56465_NTF8.tmp",
            56465.3, 56465.32, 'B1937+21, MJD 56465, 10 second subintervals')

B1937+21, MJD 56465, 10 second subintervals
sigma_all = 7.05138954322e-08
average sigma_dump = 4.92378535069e-08
jitter ratio = 1.43210742163
masked jitter ratio = 1.43210742163


In [380]:
jitter_ratio("/nimrod1/eschwab/residuals/J1022+1001_resid_57208_80F8.tmp",
            57208.778, 57208.79, 'J1022+1001, MJD 57208, 80 second subintervals')

J1022+1001, MJD 57208, 80 second subintervals
sigma_all = 4.6866049668e-05
average sigma_dump = 0.000296708047933
jitter ratio = 0.157953415806
masked jitter ratio = 3.33695161071


In [381]:
jitter_ratio("/nimrod1/eschwab/residuals/J1022+1001_resid_57208_NTF8.tmp",
            57208.778, 57208.79, 'J1022+1001, MJD 57208, 10 second subintervals')

J1022+1001, MJD 57208, 10 second subintervals
sigma_all = 0.000104350827388
average sigma_dump = 0.000345928859937
jitter ratio = 0.301654008881
masked jitter ratio = 2.1828169481
